Let's say:
- θ represents temperature
- ω represents windSpeed
- τ represents tailwindPercentage
- h represents headwindPercentage
- c represents crosswindPercentage

# $f(θ, ω, τ, h, c) = 0.4 \cdot \frac{1}{1 + e^{-(θ - 23)}} + 0.4 \cdot \frac{1}{1 + e^{-ω}} + 0.2 \cdot \frac{(τ - h - c + 100)}{200} $

In [150]:
import math

def compute_cycling_score(path_weather_impact):
    ideal_temperature = 20.0

    temperature_impact = 1 / (1 + math.exp(-(path_weather_impact['temperature'] - ideal_temperature)))

    ideal_wind_speed = 0.0

    wind_speed_impact = 1 / (1 + math.exp(-(path_weather_impact['wind_speed'] - ideal_wind_speed)))

    wind_impact = (path_weather_impact.get('tailwind_percentage', 0) - 
                   path_weather_impact.get('headwind_percentage', 0) - 
                   path_weather_impact.get('crosswind_percentage', 0))

    normalized_wind_impact = (wind_impact + 100) / 200  # normalize to 0-1 range

    temperature_weight = 0.4  # increase weight for temperature
    wind_speed_weight = 0.4  # increase weight for wind speed
    wind_impact_weight = 0.2  # reduce weight for wind impact

    score = (temperature_weight * temperature_impact + 
             wind_speed_weight * wind_speed_impact + 
             wind_impact_weight * normalized_wind_impact)

    return score

In [151]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# Create a range of temperature and wind speed values
temperatures = np.linspace(-10, 40, 100)  # temperatures from -10 to 40 degrees Celsius
wind_speeds = np.linspace(0, 20, 100)     # wind speeds from 0 to 20 m/s

# Create a 2D grid of temperature and wind speed values
temp_grid, wind_speed_grid = np.meshgrid(temperatures, wind_speeds)

# Compute the cycling score for each combination of temperature and wind speed
scores = np.zeros_like(temp_grid)
for i in range(temp_grid.shape[0]):
    for j in range(temp_grid.shape[1]):
        path_weather_impact = {
            'temperature': temp_grid[i, j],
            'wind_speed': wind_speed_grid[i, j],
            'tailwind_percentage': 20,  # keep wind impact constant
            'headwind_percentage': 10,
            'crosswind_percentage': 10,
        }
        scores[i, j] = compute_cycling_score(path_weather_impact)

In [152]:
# Flatten the 2D arrays
flat_temps = temp_grid.flatten()
flat_wind_speeds = wind_speed_grid.flatten()
flat_scores = scores.flatten()

# Create a DataFrame
df = pd.DataFrame({
    'Temperature': flat_temps,
    'Wind_Speed': flat_wind_speeds,
    'Score': flat_scores
})
import plotly.graph_objects as go
import pandas as pd
import numpy as np

# Assuming 'df' is your DataFrame
df = df.sort_values(by=['Temperature', 'Wind_Speed'])

# Reshape the 'Score' column to a 2D array
z = df['Score'].values.reshape((100, 100))

# Get the unique 'Temperature' and 'Wind_Speed' values
x = df['Temperature'].unique()
y = df['Wind_Speed'].unique()

fig = go.Figure(data=[go.Surface(z=z, x=x, y=y)])
fig.update_layout(title='Cycling Score based on Temperature and Wind Speed', autosize=False,
                  margin=dict(l=65, r=50, b=65, t=90),
                  template='plotly_dark',
                    scene=dict(xaxis_title='Temperature (°C)',
                                yaxis_title='Wind Speed (m/s)',
                                zaxis_title='Cycling Score'))

                  
fig.show()

In [153]:
import math

def find_headwind_percentage(relative_wind_angle):
    if((relative_wind_angle > 270) or (relative_wind_angle > 0 and relative_wind_angle < 90)):
        theta_rad = math.radians(relative_wind_angle)
        return int((1 + math.cos(theta_rad)) / 2 * 100)
    return 0

def find_crosswind_percentage(relative_wind_angle):
    theta_rad = math.radians(relative_wind_angle)
    return int((1 - math.cos(2 * theta_rad)) / 2 * 100)

def find_tailwind_percentage(relative_wind_angle):
    if((relative_wind_angle > 90) and (relative_wind_angle < 270)):
        theta_rad = math.radians(relative_wind_angle)
        return int((1 - math.cos(theta_rad)) / 2 * 100)
    return 0

In [154]:
import plotly.graph_objects as go

import pandas as pd
import pandas as pd
import plotly.express as px
import numpy as np

# Generate angles
angles = np.arange(0, 361, 1)

# Compute percentages
headwind_percentages = [find_headwind_percentage(angle) for angle in angles]
crosswind_percentages = [find_crosswind_percentage(angle) for angle in angles]
tailwind_percentages = [find_tailwind_percentage(angle) for angle in angles]

# Create a DataFrame
df = pd.DataFrame({
    'angles': angles,
    'headwind': headwind_percentages,
    'crosswind': crosswind_percentages,
    'tailwind': tailwind_percentages
})


fig = go.Figure()
fig.add_trace(go.Scatterpolar(
        r = df['headwind'],
        theta = df['angles'],
        mode = 'lines',
        name = 'headwind',
        line_color = 'burlywood'
    ))
fig.add_trace(go.Scatterpolar(
        r = df['crosswind'],
        theta = df['angles'],
        mode = 'lines',
        name = 'crosswind',
        line_color = 'powderblue'
    ))
fig.add_trace(go.Scatterpolar(
        r = df['tailwind'],
        theta = df['angles'],
        mode = 'lines',
        name = 'tailwind',
        line_color = 'lightcoral'
        
    ))


fig.update_layout(
    title = 'Wind Impact score',
    showlegend = True,
    template = 'plotly_dark',


)
fig.update_traces(fill='toself')
## 0 is north so change the graph to start from north
fig.update_layout(polar_angularaxis_rotation=90)


fig.show()